<a href="https://colab.research.google.com/github/SrijaG29/Movie-Data-Analysis-and-Recommendations-Using-PySpark/blob/main/Movies_ratings_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To create a project using the movie and ratings datasets with PySpark, here’s a step-by-step guide along with some ideas for questions you can answer. This will help demonstrate your PySpark skills and data analysis capabilities on your resume.

### Step 1: Data Exploration
1. **Load the datasets into PySpark**:
   - Load the movie dataset (`movies.csv`) and ratings dataset (`ratings.csv`) into PySpark DataFrames.

2. **Inspect the Data**:
   - Display the schema and first few rows of both DataFrames to understand the data structure.

### Step 2: Data Cleaning and Preparation
1. **Handle Missing Values**:
   - Check for any missing values and decide how to handle them (e.g., drop rows, fill with default values).

2. **Data Transformation**:
   - Extract the year from the movie title and create a new column `year`.
   - Split the `genres` column into an array of genres.

3. **Join the Datasets**:
   - Perform an inner join on the `movieId` column to combine the movie and ratings datasets.

### Step 3: Data Analysis
1. **Top-Rated Movies**:
   - Find the top 10 movies with the highest average rating.

2. **Popular Genres**:
   - Identify the most popular genres based on the number of ratings.

3. **User Behavior**:
   - Analyze the distribution of ratings by users. For example, find how many movies each user has rated.

4. **Yearly Trends**:
   - Determine how the average movie rating has changed over the years.

5. **Genre-Based Recommendations**:
   - For a given genre, list the top 5 movies based on average ratings.

6. **Movies with the Most Reviews**:
   - Identify the movies with the highest number of ratings.

### Step 4: Advanced Analysis
1. **User-Specific Recommendations**:
   - Build a basic recommendation system by suggesting top-rated movies that a user has not rated yet.

2. **Correlate Ratings and Release Year**:
   - Analyze if there’s any correlation between the release year and the average rating of movies.

3. **Genre Diversity in Top-Rated Movies**:
   - Examine the genre diversity among the top 100 highest-rated movies.

### Step 5: Performance Optimization
1. **Cache and Persist**:
   - Use caching and persistence in PySpark to optimize the performance of your queries.

2. **Partitioning**:
   - Apply partitioning to the data to improve the efficiency of operations, especially for large datasets.

### Step 6: Visualization
1. **Visualize Data**:
   - Use PySpark with an external library like Matplotlib, Seaborn, or even Power BI to create visualizations such as:
     - Distribution of ratings.
     - Average rating per genre.
     - Trends in movie ratings over the years.

### Conclusion
This project structure not only demonstrates your technical ability to handle and analyze data using PySpark but also your ability to derive meaningful insights from complex datasets. Once complete, you can showcase this project in your portfolio or resume, highlighting key aspects such as data cleaning, transformation, analysis, and visualization.

In [1]:
# Step 1: Download the ZIP file
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip -O movielens.zip

# Step 2: Extract the ZIP file
!unzip movielens.zip -d movielens_data


--2024-08-27 06:35:55--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘movielens.zip’

movielens.zip       100%[===================>] 955.28K  2.29MB/s    in 0.4s    

2024-08-27 06:35:56 (2.29 MB/s) - ‘movielens.zip’ saved [978202/978202]

Archive:  movielens.zip
   creating: movielens_data/ml-latest-small/
  inflating: movielens_data/ml-latest-small/links.csv  
  inflating: movielens_data/ml-latest-small/tags.csv  
  inflating: movielens_data/ml-latest-small/ratings.csv  
  inflating: movielens_data/ml-latest-small/README.txt  
  inflating: movielens_data/ml-latest-small/movies.csv  


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a331c16773a47cb7bca13fbf14d2b0dfa8585730d464edefb372faeb483ecee3
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession
    .builder
    .appName("Movie ratings")
    .master("local[*]")
    .getOrCreate()
)

In [5]:
spark

**Step 1: Data Exploration**
Load the datasets into PySpark session **"spark"**:

Load the movie dataset (movies.csv) and ratings dataset (ratings.csv) into PySpark DataFrames.
Inspect the Data:

Display the schema and first few rows of both DataFrames to understand the data structure.

In [6]:
movies_df = spark.read.format("csv").option("header",True).load("/content/movielens_data/ml-latest-small/movies.csv")
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [7]:
ratings_df = spark.read.format("csv").option("header",True).load("/content/movielens_data/ml-latest-small/ratings.csv")
ratings_df.show(truncate = False)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|1     |1      |4.0   |964982703|
|1     |3      |4.0   |964981247|
|1     |6      |4.0   |964982224|
|1     |47     |5.0   |964983815|
|1     |50     |5.0   |964982931|
|1     |70     |3.0   |964982400|
|1     |101    |5.0   |964980868|
|1     |110    |4.0   |964982176|
|1     |151    |5.0   |964984041|
|1     |157    |5.0   |964984100|
|1     |163    |5.0   |964983650|
|1     |216    |5.0   |964981208|
|1     |223    |3.0   |964980985|
|1     |231    |5.0   |964981179|
|1     |235    |4.0   |964980908|
|1     |260    |5.0   |964981680|
|1     |296    |3.0   |964982967|
|1     |316    |3.0   |964982310|
|1     |333    |5.0   |964981179|
|1     |349    |4.0   |964982563|
+------+-------+------+---------+
only showing top 20 rows



**Step 2: Data Cleaning and Preparation**

**Handle Missing Values:**

Check for any missing values and decide how to handle them (e.g., drop rows, fill with default values).

In [8]:
from pyspark.sql.functions import sum,col,when,split

In [9]:
print(movies_df.dtypes)

[('movieId', 'string'), ('title', 'string'), ('genres', 'string')]


In [10]:
print(ratings_df.dtypes)

[('userId', 'string'), ('movieId', 'string'), ('rating', 'string'), ('timestamp', 'string')]


In [11]:
from pyspark.sql import functions as F

# Create a list of expressions to count nulls in each column
null_counts_expr = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in movies_df.columns]

# Use the list in select()
null_counts = movies_df.select(*null_counts_expr)

# Show the result
null_counts.show()


+-------+-----+------+
|movieId|title|genres|
+-------+-----+------+
|      0|    0|     0|
+-------+-----+------+



In [12]:
# Create a list of expressions to count nulls in each column
null_counts_expr = [F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in ratings_df.columns]

# Use the list in select()
null_counts = ratings_df.select(*null_counts_expr)

# Show the result
null_counts.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     0|      0|     0|        0|
+------+-------+------+---------+



**" There are no null values in these datasets. "**

**Data Transformation:**

Extract the year from the movie title and create a new column year. Split the genres column into an array of genres.

In [13]:
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [14]:
total_count = movies_df.count()
print(total_count)

9742


Extracting year from title.

In [15]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

def year_extract(x):
  try:
    f = x.split('(')[-1].strip(')')
    return int(f)
  except:
    return None

x_udf = udf(year_extract,IntegerType())

movies_df = movies_df.withColumn('year',x_udf(col('title')))
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+----+
|movieId|title                                |genres                                     |year|
+-------+-------------------------------------+-------------------------------------------+----+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II (1995)   |Comedy                                     |1995|
|6      |Heat (1995)                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina (1995)                       |Comedy|Romance                             |1995|
|8      |Tom and Huck (1995)  

In [16]:
from pyspark.sql.functions import asc
# f = movies_df.orderBy(movies_df.year.desc())
f = movies_df.orderBy(movies_df.year.asc())
f.show(truncate = False)

+-------+---------------------------------------------------+-----------------------------------------+----+
|movieId|title                                              |genres                                   |year|
+-------+---------------------------------------------------+-----------------------------------------+----+
|40697  |Babylon 5                                          |Sci-Fi                                   |NULL|
|7789   |"11'09""01 - September 11 (2002)"                  |Drama                                    |NULL|
|27008  |From Dusk Till Dawn 2: Texas Blood Money (1999)    |Comedy|Crime|Horror                      |NULL|
|116668 |Dead Snow 2: Red vs. Dead (2014)                   |Action|Comedy|Horror                     |NULL|
|58404  |Justice League: The New Frontier (2008)            |Action|Adventure|Animation|Fantasy|Sci-Fi|NULL|
|58842  |Assembly (Ji jie hao) (2007)                       |Action|Drama|War                         |NULL|
|94494  |96 Minutes

Few columns are None so we will remove those columns.

In [17]:
null_count = movies_df.filter(col("year").isNull()).count()
print(null_count)

25


In [18]:
movies_df = movies_df.dropna(subset=['year'])
movies_df.show(truncate = False)

+-------+-------------------------------------+-------------------------------------------+----+
|movieId|title                                |genres                                     |year|
+-------+-------------------------------------+-------------------------------------------+----+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II (1995)   |Comedy                                     |1995|
|6      |Heat (1995)                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina (1995)                       |Comedy|Romance                             |1995|
|8      |Tom and Huck (1995)  

In [19]:
null_count = movies_df.filter(col("year").isNull()).count()
print(null_count)

0


In [20]:
from pyspark.sql.functions import asc
# f = movies_df.orderBy(movies_df.year.desc())
f = movies_df.orderBy(movies_df.year.asc())
f.show(truncate = False)

+-------+--------------------------------------------------------------------+-------------------------------+----+
|movieId|title                                                               |genres                         |year|
+-------+--------------------------------------------------------------------+-------------------------------+----+
|32898  |Trip to the Moon, A (Voyage dans la lune, Le) (1902)                |Action|Adventure|Fantasy|Sci-Fi|1902|
|49389  |The Great Train Robbery (1903)                                      |Crime|Western                  |1903|
|140541 |The Electric Hotel (1908)                                           |Animation|Comedy|Sci-Fi        |1908|
|7065   |Birth of a Nation, The (1915)                                       |Drama|War                      |1915|
|7243   |Intolerance: Love's Struggle Throughout the Ages (1916)             |Drama                          |1916|
|62383  |20,000 Leagues Under the Sea (1916)                            

In [21]:
genres_df = movies_df.select(movies_df.movieId,movies_df.genres)
genres_df.show(truncate = False)

+-------+-------------------------------------------+
|movieId|genres                                     |
+-------+-------------------------------------------+
|1      |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Adventure|Children|Fantasy                 |
|3      |Comedy|Romance                             |
|4      |Comedy|Drama|Romance                       |
|5      |Comedy                                     |
|6      |Action|Crime|Thriller                      |
|7      |Comedy|Romance                             |
|8      |Adventure|Children                         |
|9      |Action                                     |
|10     |Action|Adventure|Thriller                  |
|11     |Comedy|Drama|Romance                       |
|12     |Comedy|Horror                              |
|13     |Adventure|Animation|Children               |
|14     |Drama                                      |
|15     |Action|Adventure|Romance                   |
|16     |Crime|Drama        

In [22]:
genres_df = genres_df.withColumn("gener",split(genres_df.genres,"\|"))
genres_df.show(truncate = False)

+-------+-------------------------------------------+-------------------------------------------------+
|movieId|genres                                     |gener                                            |
+-------+-------------------------------------------+-------------------------------------------------+
|1      |Adventure|Animation|Children|Comedy|Fantasy|[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |Adventure|Children|Fantasy                 |[Adventure, Children, Fantasy]                   |
|3      |Comedy|Romance                             |[Comedy, Romance]                                |
|4      |Comedy|Drama|Romance                       |[Comedy, Drama, Romance]                         |
|5      |Comedy                                     |[Comedy]                                         |
|6      |Action|Crime|Thriller                      |[Action, Crime, Thriller]                        |
|7      |Comedy|Romance                             |[Comedy, Ro

In [23]:
genres_df = genres_df.drop(genres_df.genres)
genres_df.show(truncate = False)

+-------+-------------------------------------------------+
|movieId|gener                                            |
+-------+-------------------------------------------------+
|1      |[Adventure, Animation, Children, Comedy, Fantasy]|
|2      |[Adventure, Children, Fantasy]                   |
|3      |[Comedy, Romance]                                |
|4      |[Comedy, Drama, Romance]                         |
|5      |[Comedy]                                         |
|6      |[Action, Crime, Thriller]                        |
|7      |[Comedy, Romance]                                |
|8      |[Adventure, Children]                            |
|9      |[Action]                                         |
|10     |[Action, Adventure, Thriller]                    |
|11     |[Comedy, Drama, Romance]                         |
|12     |[Comedy, Horror]                                 |
|13     |[Adventure, Animation, Children]                 |
|14     |[Drama]                        

In movies_df there are rows in genre which have value like (no genre listed) so we will remove them.

In [24]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

def clean_genre(x):
    if x != '(no genres listed)':
        return x
    else:
        return None

clean_genre_udf = udf(clean_genre, StringType())
movies_df = movies_df.withColumn('genres', clean_genre_udf(col('genres')))
movies_df.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+----+
|movieId|title                                |genres                                     |year|
+-------+-------------------------------------+-------------------------------------------+----+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II (1995)   |Comedy                                     |1995|
|6      |Heat (1995)                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina (1995)                       |Comedy|Romance                             |1995|
|8      |Tom and Huck (1995)  

Checking for null values.

In [25]:
null_count = movies_df.filter(col("genres").isNull()).count()
print(null_count)

25


In [26]:
total_rows = movies_df.count()
print(total_rows)

9717


In [27]:
movies_df = movies_df.filter(movies_df.genres.isNotNull())
movies_df.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+----+
|movieId|title                                |genres                                     |year|
+-------+-------------------------------------+-------------------------------------------+----+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|1995|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |1995|
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |1995|
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |1995|
|5      |Father of the Bride Part II (1995)   |Comedy                                     |1995|
|6      |Heat (1995)                          |Action|Crime|Thriller                      |1995|
|7      |Sabrina (1995)                       |Comedy|Romance                             |1995|
|8      |Tom and Huck (1995)  

In [28]:
null_count = movies_df.filter(col("genres").isNull()).count()
print(null_count)

0


In [29]:
total_rows = movies_df.count()
print(total_rows)

9692


**Join the Datasets:**

Perform an inner join on the movieId column to combine the movie and ratings datasets.

In [30]:
movies_ratings_df = movies_df.join(ratings_df,on='movieId',how='inner')
movies_ratings_df.show(truncate = False)

+-------+-----------------------------------------+-------------------------------------------+----+------+------+---------+
|movieId|title                                    |genres                                     |year|userId|rating|timestamp|
+-------+-----------------------------------------+-------------------------------------------+----+------+------+---------+
|1      |Toy Story (1995)                         |Adventure|Animation|Children|Comedy|Fantasy|1995|1     |4.0   |964982703|
|3      |Grumpier Old Men (1995)                  |Comedy|Romance                             |1995|1     |4.0   |964981247|
|6      |Heat (1995)                              |Action|Crime|Thriller                      |1995|1     |4.0   |964982224|
|47     |Seven (a.k.a. Se7en) (1995)              |Mystery|Thriller                           |1995|1     |5.0   |964983815|
|50     |Usual Suspects, The (1995)               |Crime|Mystery|Thriller                     |1995|1     |5.0   |964982931|


Now we will change the data types of each column accordingly.

movieId -> int

userId -> int

ratings -> Float

In [31]:
print(movies_ratings_df.dtypes)

[('movieId', 'string'), ('title', 'string'), ('genres', 'string'), ('year', 'int'), ('userId', 'string'), ('rating', 'string'), ('timestamp', 'string')]


In [32]:
# Change the data types of multiple columns
movies_ratings_df = movies_ratings_df.withColumn("movieId", F.col("movieId").cast("integer")) \
                     .withColumn("rating", F.col("rating").cast("float"))\
                     .withColumn("userId", F.col("userId").cast("integer"))

print(movies_ratings_df.dtypes)



[('movieId', 'int'), ('title', 'string'), ('genres', 'string'), ('year', 'int'), ('userId', 'int'), ('rating', 'float'), ('timestamp', 'string')]


In [33]:
movies_ratings_df.count()

100766

**Step 3: Data Analysis**



**Top-Rated Movies:**

Find the top 10 movies with the highest average rating.

In [ ]:
from pyspark.sql.functions import avg,desc,asc
top_ten_movies = movies_ratings_df.groupby('movieId','title').agg(avg('rating').alias('Avg_rating'))
top_ten_movies = top_ten_movies.orderBy(top_ten_movies.Avg_rating.desc())
top_ten_movies.show(10,truncate = False)

+-------+-------------------------------------------------------+----------+
|movieId|title                                                  |Avg_rating|
+-------+-------------------------------------------------------+----------+
|267928 |Unreal News Reel (1923)                                |5.0       |
|189587 |The Idiot Cycle (2009)                                 |5.0       |
|240070 |SpongeBob SquarePants: Heroes of Bikini Bottom (2011)  |5.0       |
|266250 |Brian Wilson: I Just Wasn't Made for These Times (1995)|5.0       |
|221334 |Bavaria - A magical journey (2012)                     |5.0       |
|228813 |The Hero's Journey: The World of Joseph Campbell (1987)|5.0       |
|287247 |The Beach Boys: Making Pet Sounds (2017)               |5.0       |
|226832 |The Beast (2014)                                       |5.0       |
|199756 |Zum Teufel mit der Penne (1968)                        |5.0       |
|223908 |Lung (2016)                                            |5.0       |

**Popular Genres:**

Identify the most popular genres based on the number of ratings.


In [ ]:
from pyspark.sql.functions import count

popular_genres = movies_ratings_df.groupBy('genres').agg(count('rating').alias('No_of_ratings'))
popular_genres = popular_genres.sort(popular_genres.No_of_ratings.desc())
popular_genres.show(truncate = False)

+--------------------------------+-------------+
|genres                          |No_of_ratings|
+--------------------------------+-------------+
|Drama                           |2247114      |
|Comedy                          |1876631      |
|Comedy|Romance                  |1090830      |
|Drama|Romance                   |978575       |
|Comedy|Drama                    |919399       |
|Comedy|Drama|Romance            |871837       |
|Action|Adventure|Sci-Fi         |799753       |
|Crime|Drama                     |762923       |
|Action|Crime|Thriller           |482833       |
|Drama|Thriller                  |464062       |
|Action|Adventure|Sci-Fi|Thriller|438591       |
|Action|Adventure|Thriller       |415166       |
|Action|Sci-Fi|Thriller          |395940       |
|Crime|Drama|Thriller            |387572       |
|Drama|War                       |359478       |
|Action|Crime|Drama|Thriller     |327357       |
|Action|Drama|War                |325357       |
|Comedy|Crime       

**User Behavior:**

Analyze the distribution of ratings by users. For example, find how many movies each user has rated.


In [ ]:
user_rating = movies_ratings_df.groupBy('userId').agg(count('userId').alias('No_of_ratings')).orderBy(movies_ratings_df.userId.asc())
user_rating.show(truncate = False)

+------+-------------+
|userId|No_of_ratings|
+------+-------------+
|1     |141          |
|2     |52           |
|3     |147          |
|4     |27           |
|5     |33           |
|6     |26           |
|7     |44           |
|8     |30           |
|9     |58           |
|10    |660          |
|11    |20           |
|12    |23           |
|13    |65           |
|14    |29           |
|15    |82           |
|16    |294          |
|17    |86           |
|18    |138          |
|19    |47           |
|20    |139          |
+------+-------------+
only showing top 20 rows



**Yearly Trends:**

Determine how the average movie rating has changed over the years.


In [ ]:
movies_ratings_df.select('year').show()

+----+
|year|
+----+
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1995|
|1976|
|1995|
|1995|
|1995|
|1994|
|1994|
|1977|
|1994|
|1994|
|1993|
|1995|
+----+
only showing top 20 rows



In [ ]:
avg_rating_year = movies_ratings_df.groupBy('year').agg(avg('rating').alias('Avg_rating'))
avg_rating_year.show(truncate = False)

+----+------------------+
|year|Avg_rating        |
+----+------------------+
|1959|3.8107774040575713|
|1990|3.4513714113949603|
|1896|2.9234972677595628|
|1903|3.066532258064516 |
|1975|3.8815535107687804|
|1977|3.8233296303051407|
|1888|2.577272727272727 |
|1924|3.763372956909361 |
|2003|3.47684039842985  |
|2007|3.507449919061784 |
|1892|2.7777777777777777|
|2018|3.4762552780695954|
|1974|3.8950891613262746|
|2015|3.5664814492136325|
|2023|3.345118898623279 |
|1927|3.8604250713372035|
|1955|3.719717275877964 |
|1890|2.238095238095238 |
|2006|3.555690373687808 |
|1978|3.4619190067903967|
+----+------------------+
only showing top 20 rows



**Genre-Based Recommendations:**

For a given genre, list the top 5 movies based on average ratings.


In [ ]:
genre_input = input("Enter the genre to filter movies: ")

top_five_movies = (movies_ratings_df.filter(col('genres') == genre_input)\
                   .groupBy('title')\
                   .agg(avg('rating').alias('Avg_rating'))\
                   .orderBy(col('Avg_rating').desc())
                   .limit(5))
top_five_movies.show(truncate = False)


Enter the genre to filter movies: Action
+--------------------------+----------+
|title                     |Avg_rating|
+--------------------------+----------+
|Drunken Master 3 (1994)   |5.0       |
|FB: Fighting Beat (2007)  |5.0       |
|Legend of a Fighter (1982)|5.0       |
|Shanghai 13 (1984)        |5.0       |
|Street Level (2016)       |5.0       |
+--------------------------+----------+



**Movies with the Most Reviews:**

Identify the movies with the highest number of ratings.

In [ ]:
from pyspark.sql.functions import desc

In [ ]:
most_reviews = movies_ratings_df.groupBy('movieId','title')\
              .agg(count(col('rating')).alias('No_of_reviews'))\
              .orderBy(col('No_of_reviews').desc())\
              .limit(1)
most_reviews.show(truncate = False)

+-------+--------------------------------+-------------+
|movieId|title                           |No_of_reviews|
+-------+--------------------------------+-------------+
|318    |Shawshank Redemption, The (1994)|102929       |
+-------+--------------------------------+-------------+



In [ ]:
x = movies_df.filter(col("title").like("Shawshank Redemption%"))
x.show(truncate = False)

+-------+--------------------------------+-----------+----+
|movieId|title                           |genres     |year|
+-------+--------------------------------+-----------+----+
|318    |Shawshank Redemption, The (1994)|Crime|Drama|1994|
+-------+--------------------------------+-----------+----+



**Step 4: Advanced Analysis**


**User-Specific Recommendations:**

Build a basic recommendation system by suggesting top-rated movies that a user has not rated yet.


In [ ]:
top_rated_movies = movies_ratings_df.groupBy('movieId', 'title')\
    .agg(avg(col('rating')).alias('Avg_rating'))\
    .orderBy(col('Avg_rating').desc())\
    .limit(100)\
    .select('movieId','title')

top_rated_movies.show(truncate = False)

+-------+-------------------------------------------------------+
|movieId|title                                                  |
+-------+-------------------------------------------------------+
|136996 |The Wrong Girl (1999)                                  |
|117656 |Drunken Master 3 (1994)                                |
|277798 |Frank & Zed (2020)                                     |
|289775 |Toma (2021)                                            |
|226640 |Zabardast (2018)                                       |
|248740 |Legs - Atavism (2012)                                  |
|226832 |The Beast (2014)                                       |
|282565 |The Book of the Sea (2018)                             |
|266250 |Brian Wilson: I Just Wasn't Made for These Times (1995)|
|292047 |Bamboo Doll of Echizen (1963)                          |
|289249 |Cirque du Soleil: Quidam (1999)                        |
|267928 |Unreal News Reel (1923)                                |
|137018 |A

In [ ]:
total_users = movies_ratings_df.select('userId').distinct().count()
print(total_users)

200948


In [ ]:
distinct_user_ids = movies_ratings_df.select(movies_ratings_df.userId).distinct()
distinct_user_ids.show(truncate = False)

+------+
|userId|
+------+
|148   |
|463   |
|471   |
|496   |
|833   |
|1088  |
|1238  |
|1342  |
|1580  |
|1591  |
|1645  |
|1829  |
|1959  |
|2122  |
|2142  |
|2366  |
|2659  |
|2866  |
|3175  |
|3749  |
+------+
only showing top 20 rows



In [ ]:
user_id_input = input('Enter user id')

user_ratings = movies_ratings_df.filter(col('userId') == user_id_input).select('movieId','title')
user_ratings.show(truncate = False)

Enter user id1
+-------+---------------------------------------------------------------+
|movieId|title                                                          |
+-------+---------------------------------------------------------------+
|17     |Sense and Sensibility (1995)                                   |
|25     |Leaving Las Vegas (1995)                                       |
|29     |City of Lost Children, The (Cité des enfants perdus, La) (1995)|
|30     |Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)           |
|32     |Twelve Monkeys (a.k.a. 12 Monkeys) (1995)                      |
|34     |Babe (1995)                                                    |
|36     |Dead Man Walking (1995)                                        |
|80     |White Balloon, The (Badkonake sefid) (1995)                    |
|110    |Braveheart (1995)                                              |
|111    |Taxi Driver (1976)                                             |
|161    |Crimson Tide (


The **left anti join** in PySpark is similar to the join functionality, but it returns only columns from the left DataFrame for non-matched records.

In [ ]:
recommended_movies_df = top_rated_movies.join(user_ratings, on='movieId', how='left_anti')
recommended_movies_df.show(truncate = False)

+-------+-------------------------------------------------------+
|movieId|title                                                  |
+-------+-------------------------------------------------------+
|267948 |Christmas Cracker (1963)                               |
|243480 |Jealousy Is My Middle Name (2002)                      |
|247138 |The Science of Sleep (2016)                            |
|172071 |Atelier Fontana (2011)                                 |
|140369 |War Arrow (1954)                                       |
|285265 |Love in Bloom (2022)                                   |
|214182 |The Wedding Do Over (2018)                             |
|289249 |Cirque du Soleil: Quidam (1999)                        |
|257905 |The Sauce of Love (2021)                               |
|228813 |The Hero's Journey: The World of Joseph Campbell (1987)|
|143422 |2 (2007)                                               |
|282531 |A Night at the Kindergarten (2022)                     |
|267038 |C

left_anti join will keep only the movies that are not in the user’s rated movies list.

**Correlate Ratings and Release Year:**

Analyze if there’s any correlation between the release year and the average rating of movies.


In [ ]:
avg_rating_year = movies_ratings_df.groupBy('year').agg(avg(col('rating')).alias('Avg_rating'))
avg_rating_year.show(truncate = False)

+----+------------------+
|year|Avg_rating        |
+----+------------------+
|1959|3.8107774040575713|
|1990|3.4513714113949603|
|1896|2.9234972677595628|
|1903|3.066532258064516 |
|1975|3.8815535107687804|
|1977|3.8233296303051407|
|1888|2.577272727272727 |
|1924|3.763372956909361 |
|2003|3.47684039842985  |
|2007|3.507449919061784 |
|1892|2.7777777777777777|
|2018|3.4762552780695954|
|1974|3.8950891613262746|
|2015|3.5664814492136325|
|2023|3.345118898623279 |
|1927|3.8604250713372035|
|1955|3.719717275877964 |
|1890|2.238095238095238 |
|2006|3.555690373687808 |
|1978|3.4619190067903967|
+----+------------------+
only showing top 20 rows



In [ ]:
avg_rating_year_pd = avg_rating_year.toPandas()
correlation = avg_rating_year_pd['year'].astype(float).corr(avg_rating_year_pd['Avg_rating'])
print("Correlation between release year and average rating: ",correlation)


Correlation between release year and average rating:  0.3809941166120315


In this we are caluclatuing Pearson correlation coefficient between the year and Avg_rating columns.

**Conclusion:** There appears to be a moderate trend where newer movies (those released more recently) tend to receive slightly higher average ratings than older movies as pearson correlation coefficient comes under moderate strength.

**Genre Diversity in Top-Rated Movies:**

Examine the genre diversity among the top 100 highest-rated movies.

In [ ]:
top_rated_movies = movies_ratings_df.groupBy('movieId', 'title')\
    .agg(avg(col('rating')).alias('Avg_rating'))\
    .orderBy(col('Avg_rating').desc())\
    .limit(100)\
    .select('movieId','title')

top_rated_movies.show(truncate = False)

+-------+-------------------------------------------------------+
|movieId|title                                                  |
+-------+-------------------------------------------------------+
|136996 |The Wrong Girl (1999)                                  |
|117656 |Drunken Master 3 (1994)                                |
|277798 |Frank & Zed (2020)                                     |
|289775 |Toma (2021)                                            |
|226640 |Zabardast (2018)                                       |
|248740 |Legs - Atavism (2012)                                  |
|226832 |The Beast (2014)                                       |
|282565 |The Book of the Sea (2018)                             |
|266250 |Brian Wilson: I Just Wasn't Made for These Times (1995)|
|292047 |Bamboo Doll of Echizen (1963)                          |
|289249 |Cirque du Soleil: Quidam (1999)                        |
|267928 |Unreal News Reel (1923)                                |
|137018 |A

In [ ]:
from pyspark.sql.functions import explode,countDistinct

In [ ]:
movies_df_aliased = movies_df.alias('m')
top_rated_movies_aliased = top_rated_movies.alias('t')


genres_split_df = movies_df_aliased.withColumn("genre", explode(split(col("genres"), "\\|"))) \
                                   .join(top_rated_movies_aliased, on="movieId") \
                                   .select('m.title', 'genre')

genres_split_df.show(truncate=False)


+------------------------------+-----------+
|title                         |genre      |
+------------------------------+-----------+
|Fitzgerald (2002)             |Drama      |
|Until They Sail (1957)        |Drama      |
|Until They Sail (1957)        |Romance    |
|Until They Sail (1957)        |War        |
|A Sister's Revenge (2013)     |Drama      |
|A Sister's Revenge (2013)     |Mystery    |
|A Sister's Revenge (2013)     |Thriller   |
|The Perfect Neighbor (2005)   |Drama      |
|The Perfect Neighbor (2005)   |Thriller   |
|War Arrow (1954)              |Adventure  |
|War Arrow (1954)              |Drama      |
|War Arrow (1954)              |Romance    |
|War Arrow (1954)              |War        |
|War Arrow (1954)              |Western    |
|Uomo e galantuomo (1975)      |Comedy     |
|2 (2007)                      |Drama      |
|Tammy and the Doctor (1963)   |Comedy     |
|Tammy and the Doctor (1963)   |Romance    |
|Who Killed Chea Vichea? (2010)|Documentary|
|The Far P

In [ ]:
# Count distinct genres
genre_diversity = genres_split_df.groupBy("title") \
                                .agg(countDistinct("genre").alias("distinct_genres"))
genre_diversity = genre_diversity.orderBy(col('distinct_genres').desc())
genre_diversity.show(truncate = False)

+--------------------------------------------------+---------------+
|title                                             |distinct_genres|
+--------------------------------------------------+---------------+
|War Arrow (1954)                                  |5              |
|Christmas Cracker (1963)                          |4              |
|The Light in the Forest (1958)                    |4              |
|Me Sinto Bem com Você (2021)                      |3              |
|Silvery Moon (1933)                               |3              |
|Paper Marriage (1988)                             |3              |
|Until They Sail (1957)                            |3              |
|A Sister's Revenge (2013)                         |3              |
|See You Next Christmas (2021)                     |3              |
|Nico the Unicorn (1998)                           |3              |
|The Final Exit of the Disciples of Ascensia (2019)|3              |
|B&B Merry (2022)                 

**Total Genre Counts:**

You might want to count how many times each genre appears across all the top 100 movies.


In [ ]:
total_count = genres_split_df.groupBy('genre').agg(count(col('genre')).alias('No_of_count'))
total_count = total_count.orderBy(col('No_of_count').desc())
total_count.show(truncate = False)

+-----------+-----------+
|genre      |No_of_count|
+-----------+-----------+
|Romance    |52         |
|Comedy     |33         |
|Drama      |31         |
|Documentary|15         |
|Animation  |6          |
|Children   |5          |
|Action     |5          |
|Thriller   |4          |
|War        |4          |
|Horror     |4          |
|Adventure  |3          |
|Fantasy    |3          |
|Mystery    |2          |
|Western    |2          |
|Crime      |1          |
|Sci-Fi     |1          |
+-----------+-----------+



**Percentage Representation:**

Calculate the percentage representation of each genre among the top-rated movies to understand which genres dominate.

In [ ]:
from pyspark.sql.functions import round, col

percentage_count = total_count.withColumn('percentage_out_of_5',(col('No_of_count') / 5) * 100)
percentage_count.show(truncate=False)

+-----------+-----------+-------------------+
|genre      |No_of_count|percentage_out_of_5|
+-----------+-----------+-------------------+
|Romance    |52         |1040.0             |
|Comedy     |33         |660.0              |
|Drama      |31         |620.0              |
|Documentary|15         |300.0              |
|Animation  |6          |120.0              |
|Children   |5          |100.0              |
|Action     |5          |100.0              |
|Thriller   |4          |80.0               |
|War        |4          |80.0               |
|Horror     |4          |80.0               |
|Adventure  |3          |60.0               |
|Fantasy    |3          |60.0               |
|Mystery    |2          |40.0               |
|Western    |2          |40.0               |
|Crime      |1          |20.0               |
|Sci-Fi     |1          |20.0               |
+-----------+-----------+-------------------+



**Step 5: Performance Optimization**


**Cache and Persist:**

Use caching and persistence in PySpark to optimize the performance of your queries.


**Partitioning:**

Apply partitioning to the data to improve the efficiency of operations, especially for large datasets.


**Step 6: Visualization**

**Visualize Data:**
Use PySpark with an external library like Matplotlib, Seaborn, or even Power BI to create visualizations such as:
Distribution of ratings.
Average rating per genre.
Trends in movie ratings over the years.

**Pyspark MLib:**

1Q) Can we predict whether a user will give a movie a high rating (e.g., 4 or 5 stars) based on the movie's genre, year, and user's past ratings?

For this we need to do data preprocessing.

In [34]:
movies_ratings_df = movies_ratings_df.withColumn('genres',split('genres','\|'))
movies_ratings_df.show(truncate = False)

+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+
|movieId|title                                    |genres                                           |year|userId|rating|timestamp|
+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+
|1      |Toy Story (1995)                         |[Adventure, Animation, Children, Comedy, Fantasy]|1995|1     |4.0   |964982703|
|3      |Grumpier Old Men (1995)                  |[Comedy, Romance]                                |1995|1     |4.0   |964981247|
|6      |Heat (1995)                              |[Action, Crime, Thriller]                        |1995|1     |4.0   |964982224|
|47     |Seven (a.k.a. Se7en) (1995)              |[Mystery, Thriller]                              |1995|1     |5.0   |964983815|
|50     |Usual Suspects, The (1995)               |[Crime, Mystery, Thriller]      

In [35]:
from pyspark.sql.functions import explode
array_genres = movies_ratings_df.select('genres')
array_genres.show(truncate = False)

+-------------------------------------------------+
|genres                                           |
+-------------------------------------------------+
|[Adventure, Animation, Children, Comedy, Fantasy]|
|[Comedy, Romance]                                |
|[Action, Crime, Thriller]                        |
|[Mystery, Thriller]                              |
|[Crime, Mystery, Thriller]                       |
|[Action, Comedy, Horror, Thriller]               |
|[Adventure, Comedy, Crime, Romance]              |
|[Action, Drama, War]                             |
|[Action, Drama, Romance, War]                    |
|[Comedy, War]                                    |
|[Action, Romance, Western]                       |
|[Comedy]                                         |
|[Comedy]                                         |
|[Adventure, Comedy]                              |
|[Comedy, Drama]                                  |
|[Action, Adventure, Sci-Fi]                      |
|[Comedy, Cr

In [36]:
genres_exploded_df =  array_genres.withColumn('genres',explode('genres'))
genres_exploded_df.show(truncate = False)

+---------+
|genres   |
+---------+
|Adventure|
|Animation|
|Children |
|Comedy   |
|Fantasy  |
|Comedy   |
|Romance  |
|Action   |
|Crime    |
|Thriller |
|Mystery  |
|Thriller |
|Crime    |
|Mystery  |
|Thriller |
|Action   |
|Comedy   |
|Horror   |
|Thriller |
|Adventure|
+---------+
only showing top 20 rows



In [37]:
genres_exploded_df = genres_exploded_df.distinct()
genres_exploded_df.show()

+-----------+
|     genres|
+-----------+
|      Crime|
|    Romance|
|   Thriller|
|  Adventure|
|      Drama|
|        War|
|Documentary|
|    Fantasy|
|    Mystery|
|    Musical|
|  Animation|
|  Film-Noir|
|       IMAX|
|     Horror|
|    Western|
|     Comedy|
|   Children|
|     Action|
|     Sci-Fi|
+-----------+



In [38]:
genres_list = genres_exploded_df.collect()
print(genres_list[0][0])

Crime


In [39]:
l = len(genres_list)
print(l)

19


In [40]:
movies_ratings_df1 = movies_ratings_df.select('*')
movies_ratings_df1.show(truncate = False)

+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+
|movieId|title                                    |genres                                           |year|userId|rating|timestamp|
+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+
|1      |Toy Story (1995)                         |[Adventure, Animation, Children, Comedy, Fantasy]|1995|1     |4.0   |964982703|
|3      |Grumpier Old Men (1995)                  |[Comedy, Romance]                                |1995|1     |4.0   |964981247|
|6      |Heat (1995)                              |[Action, Crime, Thriller]                        |1995|1     |4.0   |964982224|
|47     |Seven (a.k.a. Se7en) (1995)              |[Mystery, Thriller]                              |1995|1     |5.0   |964983815|
|50     |Usual Suspects, The (1995)               |[Crime, Mystery, Thriller]      

In [41]:

from pyspark.sql.functions import lit

for i in range(l):
    movies_ratings_df1 = movies_ratings_df1.withColumn(str(genres_list[i][0]),lit(0))
movies_ratings_df1.show(truncate = False)


+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|movieId|title                                    |genres                                           |year|userId|rating|timestamp|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|1      |Toy Story (1995)                         |[Adventure, Animation, Children, Comedy, Fantasy]|1995|1     |4.0   |964982703|0    |0      |0       |0      

In [42]:
def genres_update(x, y):
    if y in x:
        return 1
    else:
        return 0


x_udf = udf(genres_update, IntegerType())

for i in range(l):
    genre = genres_list[i][0]
    movies_ratings_df1 = movies_ratings_df1.withColumn(genre, x_udf(col('genres'), lit(genre)))


movies_ratings_df1.show(truncate=False)

+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|movieId|title                                    |genres                                           |year|userId|rating|timestamp|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+-------+-----------------------------------------+-------------------------------------------------+----+------+------+---------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|1      |Toy Story (1995)                         |[Adventure, Animation, Children, Comedy, Fantasy]|1995|1     |4.0   |964982703|0    |0      |0       |1      

In [43]:
print(movies_ratings_df1.dtypes)

[('movieId', 'int'), ('title', 'string'), ('genres', 'array<string>'), ('year', 'int'), ('userId', 'int'), ('rating', 'float'), ('timestamp', 'string'), ('Crime', 'int'), ('Romance', 'int'), ('Thriller', 'int'), ('Adventure', 'int'), ('Drama', 'int'), ('War', 'int'), ('Documentary', 'int'), ('Fantasy', 'int'), ('Mystery', 'int'), ('Musical', 'int'), ('Animation', 'int'), ('Film-Noir', 'int'), ('IMAX', 'int'), ('Horror', 'int'), ('Western', 'int'), ('Comedy', 'int'), ('Children', 'int'), ('Action', 'int'), ('Sci-Fi', 'int')]


In [44]:
movies_ratings_df1 = movies_ratings_df1.drop('movieId','title','genres','year','timestamp')
movies_ratings_df1.show()

+------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|userId|rating|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|     1|   4.0|    0|      0|       0|        1|    0|  0|          0|      1|      0|      0|        1|        0|   0|     0|      0|     1|       1|     0|     0|
|     1|   4.0|    0|      1|       0|        0|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     1|       0|     0|     0|
|     1|   4.0|    1|      0|       1|        0|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     0|       0|     1|     0|
|     1|  

In [45]:
# print(f.dtypes)
print(genres_list)

[Row(genres='Crime'), Row(genres='Romance'), Row(genres='Thriller'), Row(genres='Adventure'), Row(genres='Drama'), Row(genres='War'), Row(genres='Documentary'), Row(genres='Fantasy'), Row(genres='Mystery'), Row(genres='Musical'), Row(genres='Animation'), Row(genres='Film-Noir'), Row(genres='IMAX'), Row(genres='Horror'), Row(genres='Western'), Row(genres='Comedy'), Row(genres='Children'), Row(genres='Action'), Row(genres='Sci-Fi')]


In [46]:
movies_ratings_df1 = movies_ratings_df1.withColumn('userId',col('userId').cast('int'))\
    .withColumn('rating',col('rating').cast('int'))

print(movies_ratings_df1.dtypes)

[('userId', 'int'), ('rating', 'int'), ('Crime', 'int'), ('Romance', 'int'), ('Thriller', 'int'), ('Adventure', 'int'), ('Drama', 'int'), ('War', 'int'), ('Documentary', 'int'), ('Fantasy', 'int'), ('Mystery', 'int'), ('Musical', 'int'), ('Animation', 'int'), ('Film-Noir', 'int'), ('IMAX', 'int'), ('Horror', 'int'), ('Western', 'int'), ('Comedy', 'int'), ('Children', 'int'), ('Action', 'int'), ('Sci-Fi', 'int')]


In [47]:
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols = ['userId','Crime','Romance','Thriller','Adventure','Drama','War','Documentary','Fantasy','Mystery','Musical','Animation','Film-Noir','IMAX','Horror','Western','Comedy','Children','Action','Sci-Fi'],outputCol = 'user_details')

movies_ratings_df1 = assembler.transform(movies_ratings_df1)

movies_ratings_df1.show(truncate = False)

+------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+-----------------------------------------------+
|userId|rating|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|user_details                                   |
+------+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+-----------------------------------------------+
|1     |4     |0    |0      |0       |1        |0    |0  |0          |1      |0      |0      |1        |0        |0   |0     |0      |1     |1       |0     |0     |(20,[0,4,8,11,16,17],[1.0,1.0,1.0,1.0,1.0,1.0])|
|1     |4     |0    |1      |0       |0        |0    |0  |0          |0      |0      |0      |0        |0        |0   |0     |0      |1     |0      

In [48]:
 movies_ratings_df1.select('user_details').show(truncate = False)

+-----------------------------------------------+
|user_details                                   |
+-----------------------------------------------+
|(20,[0,4,8,11,16,17],[1.0,1.0,1.0,1.0,1.0,1.0])|
|(20,[0,2,16],[1.0,1.0,1.0])                    |
|(20,[0,1,3,18],[1.0,1.0,1.0,1.0])              |
|(20,[0,3,9],[1.0,1.0,1.0])                     |
|(20,[0,1,3,9],[1.0,1.0,1.0,1.0])               |
|(20,[0,3,14,16,18],[1.0,1.0,1.0,1.0,1.0])      |
|(20,[0,1,2,4,16],[1.0,1.0,1.0,1.0,1.0])        |
|(20,[0,5,6,18],[1.0,1.0,1.0,1.0])              |
|(20,[0,2,5,6,18],[1.0,1.0,1.0,1.0,1.0])        |
|(20,[0,6,16],[1.0,1.0,1.0])                    |
|(20,[0,2,15,18],[1.0,1.0,1.0,1.0])             |
|(20,[0,16],[1.0,1.0])                          |
|(20,[0,16],[1.0,1.0])                          |
|(20,[0,4,16],[1.0,1.0,1.0])                    |
|(20,[0,5,16],[1.0,1.0,1.0])                    |
|(20,[0,4,18,19],[1.0,1.0,1.0,1.0])             |
|(20,[0,1,3,5,16],[1.0,1.0,1.0,1.0,1.0])        |


In [49]:
movie_data_training = movies_ratings_df1.select('rating','user_details')
movie_data_training.show(truncate = False)

+------+-----------------------------------------------+
|rating|user_details                                   |
+------+-----------------------------------------------+
|4     |(20,[0,4,8,11,16,17],[1.0,1.0,1.0,1.0,1.0,1.0])|
|4     |(20,[0,2,16],[1.0,1.0,1.0])                    |
|4     |(20,[0,1,3,18],[1.0,1.0,1.0,1.0])              |
|5     |(20,[0,3,9],[1.0,1.0,1.0])                     |
|5     |(20,[0,1,3,9],[1.0,1.0,1.0,1.0])               |
|3     |(20,[0,3,14,16,18],[1.0,1.0,1.0,1.0,1.0])      |
|5     |(20,[0,1,2,4,16],[1.0,1.0,1.0,1.0,1.0])        |
|4     |(20,[0,5,6,18],[1.0,1.0,1.0,1.0])              |
|5     |(20,[0,2,5,6,18],[1.0,1.0,1.0,1.0,1.0])        |
|5     |(20,[0,6,16],[1.0,1.0,1.0])                    |
|5     |(20,[0,2,15,18],[1.0,1.0,1.0,1.0])             |
|5     |(20,[0,16],[1.0,1.0])                          |
|3     |(20,[0,16],[1.0,1.0])                          |
|5     |(20,[0,4,16],[1.0,1.0,1.0])                    |
|4     |(20,[0,5,16],[1.0,1.0,1

In [50]:
train_data, test_data = movie_data_training.randomSplit([0.8, 0.2], seed=42)

In [62]:
train_data.show(truncate = False)
train_data.groupBy("rating").count().show()

+------+------------------------------------------------------------+
|rating|user_details                                                |
+------+------------------------------------------------------------+
|0     |(20,[0,1,2,5],[22.0,1.0,1.0,1.0])                           |
|0     |(20,[0,1,2,5],[153.0,1.0,1.0,1.0])                          |
|0     |(20,[0,1,2,5,8,9,12],[462.0,1.0,1.0,1.0,1.0,1.0,1.0])       |
|0     |(20,[0,1,2,5,16],[298.0,1.0,1.0,1.0,1.0])                   |
|0     |(20,[0,1,2,9,16],[580.0,1.0,1.0,1.0,1.0])                   |
|0     |(20,[0,1,3],[21.0,1.0,1.0])                                 |
|0     |(20,[0,1,3,4],[358.0,1.0,1.0,1.0])                          |
|0     |(20,[0,1,3,4,14,18],[22.0,1.0,1.0,1.0,1.0,1.0])             |
|0     |(20,[0,1,3,4,14,18],[41.0,1.0,1.0,1.0,1.0,1.0])             |
|0     |(20,[0,1,3,4,16,18],[274.0,1.0,1.0,1.0,1.0,1.0])            |
|0     |(20,[0,1,3,4,16,18],[438.0,1.0,1.0,1.0,1.0,1.0])            |
|0     |(20,[0,1,3,5

In [63]:
test_data.show(truncate = False)
test_data.groupBy("rating").count().show()

+------+------------------------------------------------+
|rating|user_details                                    |
+------+------------------------------------------------+
|0     |(20,[0,1,2,5],[159.0,1.0,1.0,1.0])              |
|0     |(20,[0,1,2,16],[307.0,1.0,1.0,1.0])             |
|0     |(20,[0,1,3],[517.0,1.0,1.0])                    |
|0     |(20,[0,1,3,4,16,18],[310.0,1.0,1.0,1.0,1.0,1.0])|
|0     |(20,[0,1,3,5,9],[51.0,1.0,1.0,1.0,1.0])         |
|0     |(20,[0,1,3,5,9],[298.0,1.0,1.0,1.0,1.0])        |
|0     |(20,[0,1,3,5,16],[159.0,1.0,1.0,1.0,1.0])       |
|0     |(20,[0,1,3,5,18],[393.0,1.0,1.0,1.0,1.0])       |
|0     |(20,[0,1,3,9,12,18],[393.0,1.0,1.0,1.0,1.0,1.0])|
|0     |(20,[0,1,3,9,14],[477.0,1.0,1.0,1.0,1.0])       |
|0     |(20,[0,1,3,9,14],[599.0,1.0,1.0,1.0,1.0])       |
|0     |(20,[0,1,3,9,18,19],[399.0,1.0,1.0,1.0,1.0,1.0])|
|0     |(20,[0,1,3,14],[111.0,1.0,1.0,1.0])             |
|0     |(20,[0,1,3,14],[489.0,1.0,1.0,1.0])             |
|0     |(20,[0

In [64]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol="user_details", labelCol="rating",numTrees=20)

model = rf.fit(train_data)

In [65]:
predictions = model.transform(test_data)

In [66]:
predictions.show(truncate=False)

+------+------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+----------+
|rating|user_details                                    |rawPrediction                                                                                                     |probability                                                                                                                |prediction|
+------+------------------------------------------------+------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+----------+
|0     |(20,[0,1,2,5],[159.0,1.0,1.0,1.0])              |[0.2084878416278130

In [67]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       4.0|12768|
|       3.0| 7471|
+----------+-----+



In [69]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="rating",
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.37926775038292404


In [78]:
from pyspark.ml.regression import RandomForestRegressor
dt = RandomForestRegressor(
    featuresCol="user_details",
    labelCol="rating"
)

dt_model = dt.fit(train_data)

In [79]:
dt_predictions =dt_model.transform(test_data)

In [81]:
dt_predictions.show()

+------+--------------------+------------------+
|rating|        user_details|        prediction|
+------+--------------------+------------------+
|     0|(20,[0,1,2,5],[15...|3.5994130115648915|
|     0|(20,[0,1,2,16],[3...| 3.145741202671794|
|     0|(20,[0,1,3],[517....|3.4220843522784725|
|     0|(20,[0,1,3,4,16,1...|3.1235446137990985|
|     0|(20,[0,1,3,5,9],[...|3.5500368153389963|
|     0|(20,[0,1,3,5,9],[...| 3.533358205967648|
|     0|(20,[0,1,3,5,16],...| 3.617680791464584|
|     0|(20,[0,1,3,5,18],...|3.5007354083360864|
|     0|(20,[0,1,3,9,12,1...|3.4708277021634637|
|     0|(20,[0,1,3,9,14],...|  3.43574196091336|
|     0|(20,[0,1,3,9,14],...|3.2294246873310684|
|     0|(20,[0,1,3,9,18,1...| 3.400370448785888|
|     0|(20,[0,1,3,14],[1...|3.4306476245509336|
|     0|(20,[0,1,3,14],[4...| 3.439115334929464|
|     0|(20,[0,1,3,16,18]...|3.0888253269171266|
|     0|(20,[0,1,3,18],[1...| 3.322083210797575|
|     0|(20,[0,1,3,18],[5...|3.3072621731302894|
|     0|(20,[0,1,5,1

In [80]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol="rating",
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy = evaluator.evaluate(dt_predictions)
print(accuracy)

0.0


In [66]:
model.save("logistic_regression_recommendation")

In [67]:
from pyspark.ml.classification import LogisticRegressionModel
model1 = LogisticRegressionModel.load("/content/logistic_regression_recommendation")

In [68]:
user_id = 148
generes = ['Action','Adventure','Thriller']

In [69]:
d ={}

for i in range(l):
    d[genres_list[i][0]] = i+1

print(d)



{'Crime': 1, 'Romance': 2, 'Thriller': 3, 'Adventure': 4, 'Drama': 5, 'War': 6, 'Documentary': 7, 'Fantasy': 8, 'Mystery': 9, 'Musical': 10, 'Animation': 11, 'Film-Noir': 12, 'IMAX': 13, 'Horror': 14, 'Western': 15, 'Comedy': 16, 'Children': 17, 'Action': 18, 'Sci-Fi': 19}


In [70]:
details = [0]*20
print(details)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [71]:
details[0] = 148

for i in generes:
    details[d.get(i)] = 1

print(details)

[148, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [78]:
from pyspark.sql.types import IntegerType,StructType,StructField
schema = StructType(
    [
        StructField('userId',IntegerType()),
        StructField('Crime',IntegerType()),
        StructField('Romance',IntegerType()),
        StructField('Thriller',IntegerType()),
        StructField('Adventure',IntegerType()),
        StructField('Drama',IntegerType()),
        StructField('War',IntegerType()),
        StructField('Documentary',IntegerType()),
        StructField('Fantasy',IntegerType()),
        StructField('Mystery',IntegerType()),
        StructField('Musical',IntegerType()),
        StructField('Animation',IntegerType()),
        StructField('Film-Noir',IntegerType()),
        StructField('IMAX',IntegerType()),
        StructField('Horror',IntegerType()),
        StructField('Western',IntegerType()),
        StructField('Comedy',IntegerType()),
        StructField('Children',IntegerType()),
        StructField('Action',IntegerType()),
        StructField('Sci-Fi',IntegerType())
    ]
)

In [85]:
data = tuple(details)
print(data)

(148, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)


In [86]:
df = spark.createDataFrame([data],schema=schema)
df.show()

+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|userId|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|   148|    0|      0|       1|        1|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     0|       0|     1|     0|
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+



In [87]:
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols = ['userId','Crime','Romance','Thriller','Adventure','Drama','War','Documentary','Fantasy','Mystery','Musical','Animation','Film-Noir','IMAX','Horror','Western','Comedy','Children','Action','Sci-Fi'],outputCol = 'user_details')

df = assembler.transform(df)

df.show(truncate = False)

+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+-----------------------------------+
|userId|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|user_details                       |
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+-----------------------------------+
|148   |0    |0      |1       |1        |0    |0  |0          |0      |0      |0      |0        |0        |0   |0     |0      |0     |0       |1     |0     |(20,[0,3,4,18],[148.0,1.0,1.0,1.0])|
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+-----------------------------------+



In [88]:
x = df.select('user_details')
x.show()

+--------------------+
|        user_details|
+--------------------+
|(20,[0,3,4,18],[1...|
+--------------------+



In [89]:
prediction_1 = model1.transform(x)

In [90]:
prediction_1.show(truncate = False)

+-----------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+----------+
|user_details                       |rawPrediction                                                                                                        |probability                                                                                                          |prediction|
+-----------------------------------+---------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------+----------+
|(20,[0,3,4,18],[148.0,1.0,1.0,1.0])|[-1.84208195762524,-0.6892472122680454,0.2508165899346523,1.217662518368797,1.0672835565617929,-0.0044334949

In [91]:
unique_user_id = movies_ratings_df.select(movies_ratings_df.userId)
unique_user_id.show()

+------+
|userId|
+------+
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
+------+
only showing top 20 rows



In [92]:
unique_user_id = unique_user_id.distinct()
unique_user_id.show()

+------+
|userId|
+------+
|   148|
|   463|
|   471|
|   496|
|   833|
|  1088|
|  1238|
|  1342|
|  1580|
|  1591|
|  1645|
|  1829|
|  1959|
|  2122|
|  2142|
|  2366|
|  2659|
|  2866|
|  3175|
|  3749|
+------+
only showing top 20 rows



In [93]:
users_list = unique_user_id.collect()

In [94]:
ul = len(users_list)
print(ul)

200948


In [97]:
details = [0]*20
for i in generes:
    details[d.get(i)] = 1

print(details)


[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [100]:
x = []
for i in users_list:
  details[0] = i[0]
  x.append(tuple(details))


In [102]:
print(x[1])

(463, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0)


In [103]:
df = spark.createDataFrame(x,schema=schema)
df.show()

+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|userId|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+
|   148|    0|      0|       1|        1|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     0|       0|     1|     0|
|   463|    0|      0|       1|        1|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     0|       0|     1|     0|
|   471|    0|      0|       1|        1|    0|  0|          0|      0|      0|      0|        0|        0|   0|     0|      0|     0|       0|     1|     0|
|   496|    0|      0|       1|        1|    0|  0| 

In [104]:
from pyspark.ml.feature import VectorAssembler


assembler = VectorAssembler(inputCols = ['userId','Crime','Romance','Thriller','Adventure','Drama','War','Documentary','Fantasy','Mystery','Musical','Animation','Film-Noir','IMAX','Horror','Western','Comedy','Children','Action','Sci-Fi'],outputCol = 'user_details')

df = assembler.transform(df)

df.show(truncate = False)

+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+------------------------------------+
|userId|Crime|Romance|Thriller|Adventure|Drama|War|Documentary|Fantasy|Mystery|Musical|Animation|Film-Noir|IMAX|Horror|Western|Comedy|Children|Action|Sci-Fi|user_details                        |
+------+-----+-------+--------+---------+-----+---+-----------+-------+-------+-------+---------+---------+----+------+-------+------+--------+------+------+------------------------------------+
|148   |0    |0      |1       |1        |0    |0  |0          |0      |0      |0      |0        |0        |0   |0     |0      |0     |0       |1     |0     |(20,[0,3,4,18],[148.0,1.0,1.0,1.0]) |
|463   |0    |0      |1       |1        |0    |0  |0          |0      |0      |0      |0        |0        |0   |0     |0      |0     |0       |1     |0     |(20,[0,3,4,18],[463.0,1.0,1.0,1.0]) |
|471   |0    |0      |1  

In [105]:
df = df.select(df.user_details)
df.show(truncate = False)

+------------------------------------+
|user_details                        |
+------------------------------------+
|(20,[0,3,4,18],[148.0,1.0,1.0,1.0]) |
|(20,[0,3,4,18],[463.0,1.0,1.0,1.0]) |
|(20,[0,3,4,18],[471.0,1.0,1.0,1.0]) |
|(20,[0,3,4,18],[496.0,1.0,1.0,1.0]) |
|(20,[0,3,4,18],[833.0,1.0,1.0,1.0]) |
|(20,[0,3,4,18],[1088.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1238.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1342.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1580.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1591.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1645.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1829.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[1959.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[2122.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[2142.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[2366.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[2659.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[2866.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[3175.0,1.0,1.0,1.0])|
|(20,[0,3,4,18],[3749.0,1.0,1.0,1.0])|
+------------------------------------+
only showing top 20 rows



In [106]:
prediction_2 = model1.transform(df)

In [107]:
prediction_2.show(truncate = False)

+------------------------------------+-------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+----------+
|user_details                        |rawPrediction                                                                                                            |probability                                                                                                               |prediction|
+------------------------------------+-------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------+----------+
|(20,[0,3,4,18],[148.0,1.0,1.0,1.0]) |[-1.84208195762524,-0.6892472122680454,0.2508165899346523,1.217662518368797,1

In [111]:
prediction_2.filter(prediction_2.prediction == 1.0).show()

+------------+-------------+-----------+----------+
|user_details|rawPrediction|probability|prediction|
+------------+-------------+-----------+----------+
+------------+-------------+-----------+----------+



In [112]:
prediction_2.groupBy('prediction').count().show()

+----------+------+
|prediction| count|
+----------+------+
|       3.0|200948|
+----------+------+

